In [ ]:
import numpy as np
from numpy import sqrt, pi
import math
import matplotlib.pyplot as plt

import plotly.express as px
import plotly.graph_objects as go

import cadquery as cq
from cadquery import exporters
from jupyter_cadquery.cadquery import (PartGroup, Part, Edges, 
                                       Faces, Vertices, show)
from jupyter_cadquery import set_defaults, set_sidecar
set_defaults(theme="dark")
set_sidecar("CadQuery", init=True)

from tqdm import tqdm
from time import perf_counter

import os
from pathlib import Path
cwd=os.getcwd()
home = str(Path(os.path.abspath(cwd)).parents[1])

import pandas as pd

In [ ]:
# FUNCTIONS

In [ ]:
def hexGen(xVect,yVect):
    for i in range(len(yVect)):
        if i==0:
            yNew=np.ones(len(xVect))*yVect[i]
            xNew=xVect
        else:
            yNew=np.concatenate((yNew, np.ones(len(xVect))*yVect[i]),
                                axis=None)
            xNew=np.concatenate((xNew, xVect), axis=None)
    return xNew, yNew

In [ ]:
def hedronGen(quadside):

    a=sqrt(3)*quadside/2 #edge length
    L=quadside/2
    pts = [
        (L,L),
        (-L,L),
        (-L,-L),
        (L,-L)
    ]
    workplane=cq.Workplane('XY').transformed(offset=cq.Vector(0, 0, -L))
    base=workplane.polyline(pts).close().extrude(quadside)
    
    dx=quadside #in fact dy
    dz=quadside #in fact dx
    dy=quadside/2 #in fact dz
    wedge1=workplane.transformed(offset=cq.Vector(0,0,dy/2+quadside),
                     rotate=cq.Vector(90,0,0)).wedge(dx, dy, dz, dx/2,
                     dz/2, dx/2, dz/2, pnt=cq.Vector(0.0, 0.0, 0.0),
                     dir=cq.Vector(0.0, 0.0, 1.0))
    wedge2=workplane.workplane(invert=True).transformed(offset=
                     cq.Vector(0,0,dy/2),rotate=cq.Vector(90,0,
                     0)).wedge(dx, dy,dz, dx/2, dz/2, dx/2, dz/2, pnt=
                     cq.Vector(0.0, 0.0, 0.0), dir=cq.Vector(0.0, 0.0,
                     1.0))
    wedge3=workplane.transformed(offset=cq.Vector(0,3*dy/2,quadside/2),
                     rotate=cq.Vector(0,90,0)).wedge(dx, dy, dz, 
                     dx/2, dz/2, dx/2, dz/2, pnt=cq.Vector(0.0, 
                     0.0, 0.0), dir=cq.Vector(0.0, 0.0, 1.0))
    wedge4=workplane.transformed(offset=cq.Vector(0,-quadside/2-dy/2,
                     quadside/2),rotate=cq.Vector(0,90,0)).workplane(
                     invert=True).wedge(dx, dy, dz, dx/2, dz/2, 
                     dx/2, dz/2, pnt=cq.Vector(0.0, 0.0, 0.0), 
                     dir=cq.Vector(0.0, 0.0, 1.0))
    wedge5=workplane.transformed(rotate=cq.Vector(0,0,90)).transformed(
                     offset=cq.Vector(0,quadside/2+dy/2,
                     quadside/2),rotate=cq.Vector(0,90,0)).wedge(
                     dx, dy, dz, dx/2, dz/2, dx/2, dz/2, pnt=
                     cq.Vector(0.0, 0.0, 0.0), dir=cq.Vector(
                     0.0, 0.0, 1.0))
    wedge6=workplane.transformed(rotate=cq.Vector(0,0,90)).transformed(
                     offset=cq.Vector(0,dy/2-quadside,
                     quadside/2),rotate=cq.Vector(0,90,
                     0)).workplane(invert=True).wedge(dx, dy, dz,
                     dx/2, dz/2, dx/2, dz/2, pnt=cq.Vector(0.0, 
                     0.0, 0.0), dir=cq.Vector(0.0, 0.0, 1.0))
    
    rhombicDodecahedron=(base.union(wedge1,glue=True,clean=True).union(
                wedge2,glue=True,clean=True).union(wedge3,glue=True,
                clean=True).union(wedge4,glue=True,clean=True
                ).union(wedge5,glue=True,clean=True).union(wedge6,
                glue=True, clean=True))
    
    return rhombicDodecahedron

In [ ]:
def abcGen(L,T,a1,X1,X2,Y1,Y2,Z1,Z2,A1,A2):
    sD=cq.Workplane('XY').transformed(offset=cq.Vector(0, 0,
        Z1[0])).rect(L,T,centered=True,forConstruction=
        False).extrude(a1)
    
    # the box is ready
    hedra=[]
    
    for i in tqdm(range(X1.size)):
        hedra.append(hedronGen(A1[i]).translate(cq.Vector(X1[i],Y1[i],
                                                          Z1[i])))

    for i in tqdm(range(X2.size)):
        hedra.append(hedronGen(A2[i]).translate(cq.Vector(X2[i],Y2[i],
                                                          Z2[i])))
    
    for i in tqdm(range(len(hedra))):
        sD=sD.cut(hedra[i],clean=True)

    return sD

In [ ]:
def densify(R,VF,d,v):
    volFrac=np.ones(len(R))
    for i in range(len(R)):
        if R[i]<=(d/2):
            volFrac[i]=1
        elif (R[i]>=(d/2) and R[i]<=(d/2+v)):
            volFrac[i]=((VF-1)*pow(v,-3)*(-2*pow(R[i],3)+3*(d+v)*pow(
            R[i],2)-((3*d*(d+2*v))/2)*R[i]+((pow(d,2)*(d+3*v))/4))+1)
        else:
            volFrac[i]=VF

    return volFrac

In [ ]:
def calcSideLen(volFrac,a1):
    A=np.zeros(len(volFrac))
    for i in range(len(volFrac)):
        A[i]=a1*((1-volFrac[i])**(1./3.))
    return A

In [ ]:
def calcDist(X,Y):
    R=np.zeros(len(X))
    for i in range(len(R)):
        R[i]=sqrt(X[i]*X[i]+Y[i]*Y[i])
    return R

In [ ]:
def remPts(X,Y,Z,R,vF,A,d):
    rem_ind=[]
    for i in range(len(R)):
        if (R[i]-A[i])<(d/2) or A[i]<=0.01:
            rem_ind.append(i)
    rem_ind.reverse()
    for i in range(len(rem_ind)):
        X=np.delete(X,rem_ind[i])
        Y=np.delete(Y,rem_ind[i])
        Z=np.delete(Z,rem_ind[i])
        R=np.delete(R,rem_ind[i])
        vF=np.delete(vF,rem_ind[i])
        A=np.delete(A,rem_ind[i])
    return X, Y, Z, R, vF, A

In [ ]:
def speak(text):
    from IPython.display import Javascript as js, clear_output
    # Escape single quotes
    text = text.replace("'", r"\'")
    display(js(f'''
    if(window.speechSynthesis) {{
        var synth = window.speechSynthesis;
        synth.speak(new window.SpeechSynthesisUtterance('{text}'));
    }}
    '''))
    # Clear the JS so the notebook doesn't speak again when reopened
    clear_output(False)

In [ ]:
# DATA
def generateStructure(Name,VF,minCellSize,maxCellSize,L,T,U,implSize,
                      holeDiam,v,isoGen):
    
    #based on Min and Max Cell Size, sphere equivalent
    a0=((minCellSize+maxCellSize)/4)*((2*math.pi/3)**(1./3.))
    a1=a0/((1-VF)**(1./3.))
    
    L=math.ceil(L/(2*a1))*2*a1
    T=math.ceil(T/(2*a1))*2*a1
    U=math.ceil(U/(2*a1))*2*a1
    
    if isoGen:
        fileNameSpecifyer=(str(Name)+"_VF"+str(VF)+"_CS"
        +"{:.1f}".format(minCellSize)+"-"
        +"{:.1f}".format(maxCellSize)+"_L"
        +"{:.4f}".format(L)+"_T"+"{:.4f}".format(T)+"_U"
        +"{:.4f}".format(U)+"_ISO")
    else:
        fileNameSpecifyer=(str(Name)+"_VF"+str(VF)+"_CS"
        +"{:.1f}".format(minCellSize)+"-"
        +"{:.1f}".format(maxCellSize)+"_L"
        +"{:.4f}".format(L)+"_T"+"{:.4f}".format(T)+"_U"
        +"{:.4f}".format(U))
    
    D=2*a1 #Distance between Centres on the x axis
    H=a1 #Distance between Centres on the y axis
    
    Nxa=math.ceil(L/D+1)
    if Nxa%2 != 1: #if not odd
        Nxb=Nxa+2 #then with +2 it is good for b-row
        Nxa=Nxa+1 #and has to be incremented for a-row
    else: #if odd
        Nxb=Nxa+1 #then a is good, b must be incremented
        
    Ny=math.ceil(T/H+1)
    if Ny%2 != 1: #if not odd
        Ny=Ny+1 #then increment it
    if ((Ny-1)/2)%2 != 1: #if not odd
        Nya=int((Ny+1)/2)
        Nyb=int(Nya+1)
    else:
        Nyb=int((Ny+1)/2)
        Nya=int(Nyb+1)
    Ny=Nya+Nyb
    print("a0 = ",a0,"\nL = ",L,"\nT = ",T,"\nU = ",U,"\nD = ", D, 
          "\nH = ", H, "\nNx:\n\tNxa = ", Nxa, "\n\tNxb = ", Nxb,
          "\nNy = ", Ny, "\n\tNya = ", Nya, "\n\tNyb = ", Nyb)
    
    # LAYER 'A'
    aX=np.linspace(-((Nxa-1)*D)/2, ((Nxa-1)*D)/2, num=Nxa)
    bX=np.linspace(-((Nxa-1)*D)/2-(D/2), ((Nxa-1)*D)/2+(D/2), num=Nxb)
    
    if Nya>Nyb:
        aY=np.linspace(-((Nya+Nyb-1)*H)/2, ((Nya+Nyb-1)*H)/2, num=Nya)
        bY=np.linspace(-((Nya+Nyb-1)*H)/2+H,
                       ((Nya+Nyb-1)*H)/2-H, num=Nyb)
    else:
        bY=np.linspace(-((Nya+Nyb-1)*H)/2, ((Nya+Nyb-1)*H)/2, num=Nyb)
        aY=np.linspace(-((Nya+Nyb-1)*H)/2+H, 
                       ((Nya+Nyb-1)*H)/2-H, num=Nya)
    
    aX0=aX
    aY0=aY
    bX0=bX
    bY0=bY
    AaX, AaY = hexGen(aX0,aY0)
    AbX, AbY = hexGen(bX0,bY0)
    
    # LAYER 'B'
    BaX=AaX
    BaY=AaY-a1
    BbX=AbX
    BbY=AbY-a1
    
    if Nya>Nyb: #'a' row is on the outside
        BbX=np.concatenate((BbX, bX0), axis=None)
        BbY=np.concatenate((BbY, np.ones(len(bX0))*(BaY[-1]+H)), 
                           axis=None)
            
    else: #'b' row is on the outside
        BaX=np.concatenate((BaX, aX0), axis=None)
        BaY=np.concatenate((BaY, np.ones(len(aX0))*(BbY[-1]+H)), 
                           axis=None)
        
    AX=np.concatenate((AaX, AbX), axis=None)
    AY=np.concatenate((AaY, AbY), axis=None)
    BX=np.concatenate((BaX, BbX), axis=None)
    BY=np.concatenate((BaY, BbY), axis=None)
    
    AZ=np.zeros((len(AX)))
    BZ=np.ones((len(BX)))*a1
    
    X=np.concatenate((AX,BX), axis=None)
    Y=np.concatenate((AY,BY), axis=None)
    
    R=calcDist(X,Y)
        
    if not isoGen:
        volFrac=densify(R,VF,holeDiam,v)
    else:
        volFrac=np.ones(len(R))*VF #turn on for isotropic test body gen
    
    A=calcSideLen(volFrac,a1)
    
    fig = px.scatter(x=X, y=Y, color=volFrac,
                 labels={"color":"Volume Fraction"},
                 title="Volume Fraction",width=700,height=700,
                 color_continuous_scale='Plasma')
    fig.show()
    
    RA=calcDist(AX,AY)
    RB=calcDist(BX,BY)
    
    if not isoGen:    
        volFracA=densify(RA,VF,holeDiam,v)
        volFracB=densify(RB,VF,holeDiam,v)
    
        AA=calcSideLen(volFracA,a1)
        AB=calcSideLen(volFracB,a1)
        
        AX, AY, AZ, RA, volFracA, AA = remPts(AX, AY, AZ, RA, 
                                          volFracA, AA, implSize)
        BX, BY, BZ, RB, volFracB, AB = remPts(BX, BY, BZ, RB, 
                                          volFracB, AB, implSize)
        
    else:
        volFracA=np.ones(len(RA))*VF
        volFracB=np.ones(len(RB))*VF
        AA=calcSideLen(volFracA,a1)
        AB=calcSideLen(volFracB,a1)
            
    AZ1=AZ
    AZ2=AZ+2*a1
    sDA=abcGen(L,T,a1,AX,BX,AY,BY,AZ1,BZ,AA,AB)
    show(sDA, grid=True)
        
    sDB=abcGen(L,T,a1,BX,AX,BY,AY,BZ,AZ2,AB,AA)
    show(sDB, grid=True)
        
    tic = perf_counter()
    sDAB=sDA.union(sDB,glue=True)
    show(sDAB,grid=True)
    tac = perf_counter()
    print("Elapsed time:", '%.5f' % (tac-tic) , "s")
        
    Zsteps=math.ceil(math.log((U/(a1)), 2))
    Zmax=pow(2,Zsteps-1)*2*a1
    diffZ=Zmax-U
    
    tic = perf_counter()
    for i in tqdm(range(Zsteps)):
        if i==0:
            sDGlued=sDAB
        else:
            sDGlued=sDGlued.union(sDGlued.translate(cq.Vector(0,0,
                                        pow(2,i-1)*2*a1)),glue=True)
    show(sDGlued,grid=True)
    tac = perf_counter()
    speak("Glued all layers.")
    print("Elapsed time:", '%.5f' % (tac-tic) , "s")
    
    if not isoGen:
        tic = perf_counter()
        exporters.export(sDGlued, home+'\\out\\step\\BAM_sDGlued'
                         +fileNameSpecifyer+'.step')
        tac = perf_counter()
        print("Elapsed time:", '%.5f' % (tac-tic) , "s")
        
    tic = perf_counter()
    slicer=cq.Workplane('XY').transformed(offset=cq.Vector(0,0,U
                                          )).rect(L+1,T+1,
                                          centered=True,forConstruction=
                                          False).extrude(diffZ+1)
    sDFinal=sDGlued.cut(slicer,clean=True)
    show(sDFinal)
    tac = perf_counter()
    speak("Cut model to size.")
    print("Elapsed time:", '%.5f' % (tac-tic) , "s")
    
    tic = perf_counter()
    exporters.export(sDFinal, home+'\\out\\step\\BAM_sDFinal'
                     +fileNameSpecifyer+'.step')
    tac = perf_counter()
    print("Elapsed time:", '%.5f' % (tac-tic) , "s")
    
    if not isoGen:    
        tic = perf_counter()
        hole=cq.Workplane('XY').circle(holeDiam/2).extrude(U)
        sDFinal=sDFinal.cut(hole,clean=True)
        show(sDFinal)
        tac = perf_counter()
        speak("Drilled the hole.")
        print("Elapsed time:", '%.5f' % (tac-tic) , "s")
        
    if not isoGen:
        tic = perf_counter()
        exporters.export(sDFinal, home+'\\out\\step\\BAM_sDFinal_wHole'
                         +fileNameSpecifyer+'.step')
        tac = perf_counter()
        print("Elapsed time:", '%.5f' % (tac-tic) , "s")
        
    speak("Model done.")

In [ ]:
df = pd.read_csv(home+'\\src\\csv\\inputData.csv')
print(df)

for i in range(df.shape[0]):
    Name=df.at[i,'Name']
    VF=df.at[i,'VF']
    minCellSize=df.at[i,'minCellSize']
    maxCellSize=df.at[i,'maxCellSize']
    L=df.at[i,'L']
    T=df.at[i,'T']
    U=df.at[i,'U']
    isoGen=df.at[i,'isoGen']
    if not isoGen:
        implSize=df.at[i,'implSize']
        holeDiam=df.at[i,'holeDiam']
        v=df.loc[i].v
        generateStructure(Name,VF,minCellSize,maxCellSize,L,T,U,
                          implSize,holeDiam,v,isoGen)
    else:
        generateStructure(Name,VF,minCellSize,maxCellSize,L,T,U,
                          implSize,0,0,isoGen)